# Dependency Parsing demo

## Universal Dependencies


- advmod => adverbial modifier (modificador adverbial)
- nsubj => nominal subject (sujeto nominal)
- dobj  => direct object (objeto directo)
- iobj  => indirect object (objeto indirecto)
- nummod => numeric modifier (modificador numeral)
- appos => appositional modifier (modificador aposicional)
- nmod => nominal modifier (modificador nominal)

(...)

<center><img src="ud.png" width="600"/></center>


Más información:
- https://universaldependencies.org/u/dep/
- https://nlp.stanford.edu/pubs/USD_LREC14_paper_camera_ready.pdf

### Objetivo: Representacion compatible entre diferentes lenguajes

## Representación de los textos

### Texto base 

In [2]:
file_path = "/home/droca1/Treebanks/corpus-eusebio.txt"
data_file = open(file_path, "r")
data = data_file.read()

print(data[:1000])

1
En el texto, la autora habla de lo que son las bellísimas personas. Las define como gente que se
calla lo que piensa por no discutir, por no crear discordia. Ella vivió rodeada por bellísimas
personas, más ella no se considera parte de ellas. No tiene miedo de morderse la lengua para
expresar su opinión. En el texto se pregunta si sería capaz de trabar amistad con gente que tiene
ideales opuestos a los suyos, ideales que ella consideraría "estúpidos". ¿Sería yo amiga de
alguien con ideas distintas a las mías?

No viví rodeada de bellísimas personas, como la autora. Mi entorno han sido personas que
sabían cuando debían callar y cuando hablar y dar su opinión. Personas que no se callaban lo
que pensaban solo para gustar más, caer mejor a la gente o evitar disputas. Personas que van sin
filtros, que te dicen las cosas como son. Personas que van con la más cruda verdad por delante,
aunque duela, aunque lastime.

Me gusta imaginarme que soy una de ellos, pero la presión social hay veces q

### Representacion de dependencias

<center><img src="demo.png" width="1200"/></center>

### Representacion para el ordenador (Universal Dependencies formato CONLL-U)

In [5]:
conllu_file_path = "./demo.conllu"
conllu_file = open(conllu_file_path, "r")

# read until the first empty line
conllu = conllu_file.read()
conllu = conllu.split("\n\n")
conllu = conllu[0]

print(conllu[:1000])

# text = En el texto , la autora habla de lo que son las bellísimas personas .
1	En	_	_	_	_	3	case	_	_
2	el	_	_	_	_	3	det	_	_
3	texto	_	_	_	_	7	obl	_	_
4	,	_	_	_	_	3	punct	_	_
5	la	_	_	_	_	6	det	_	_
6	autora	_	_	_	_	7	nsubj	_	_
7	habla	_	_	_	_	0	root	_	_
8	de	_	_	_	_	14	mark	_	_
9	lo	_	_	_	_	14	det	_	_
10	que	_	_	_	_	14	nsubj	_	_
11	son	_	_	_	_	14	cop	_	_
12	las	_	_	_	_	14	det	_	_
13	bellísimas	_	_	_	_	14	amod	_	_
14	personas	_	_	_	_	7	ccomp	_	_
15	.	_	_	_	_	7	punct	_	_


## Como pasar de texto a dependencias?

### 1 - Preprocesar las frases
- Dejar una linea por frase
- Separar los signos de puntuacion de las palabras
- Eliminar espacios

In [33]:
import re

file_path = "/home/droca1/Treebanks/corpus-eusebio.txt"
data_file = open(file_path, "r")
data = data_file.read()

# remove <number>\n using regex
data = re.sub(r'\d+\n', '', data)

# remove \n
data = data.replace("\n", " ")

# split into sentences using regex
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', data)

# sentences = [s + "." for s in sentences]
sentences = [s.strip() for s in sentences]

# close characters
punctuation = [".", ",", ";", ":", "!", "?", "(", ")", "[", "]", "{", "}", "”"]
for p in punctuation:
    sentences = [s.replace(p, " " + p) for s in sentences]

# open characters
punctuation = ["¿", "¡", "“"]
for p in punctuation:
    sentences = [s.replace(p, p + " ") for s in sentences]

# bidirectional quotes
punctuation = ["\"", "'"]
for p in punctuation:
    sentences = [s.replace(p, " " + p + " ") for s in sentences]

# remove extra spaces
sentences = [re.sub(' +', ' ', s) for s in sentences]

for s in sentences[:10]:
    print(s)

# write adding a root token
output_file = "./demo.raw"
with open(output_file, "w") as f:
    for sent in sentences:
        f.write("-ROOT- " + sent + "\n")

En el texto , la autora habla de lo que son las bellísimas personas .
Las define como gente que se calla lo que piensa por no discutir , por no crear discordia .
Ella vivió rodeada por bellísimas personas , más ella no se considera parte de ellas .
No tiene miedo de morderse la lengua para expresar su opinión .
En el texto se pregunta si sería capaz de trabar amistad con gente que tiene ideales opuestos a los suyos , ideales que ella consideraría " estúpidos " .
¿ Sería yo amiga de alguien con ideas distintas a las mías ?
No viví rodeada de bellísimas personas , como la autora .
Mi entorno han sido personas que sabían cuando debían callar y cuando hablar y dar su opinión .
Personas que no se callaban lo que pensaban solo para gustar más , caer mejor a la gente o evitar disputas .
Personas que van sin filtros , que te dicen las cosas como son .


### 2 - Usar el modelo pre-entrenado para obtener las representaciones intermedias

Usar un modelo que haya aprendido las etiquetas que representen una abstracción de las relaciones de dependencia entre palabras para asignarselas a las frases. El modelo ha sido entrenado usando https://universaldependencies.org/treebanks/es_ancora/index.html

In [ ]:
# python ~/machamp/predict.py ./spanish-model/model.pt ./demo.raw ./demo.labels --raw_text

In [7]:
postprocess_path = "./demo.labels"
postprocess_file = open(postprocess_path, "r")
postprocess = postprocess_file.read()
# read until empty line
postprocess = postprocess.split("\n\n")[0]
print(postprocess[:1000])

1	-ROOT-	/0	-NOREL-	_	_	_	_	_	_
2	En	<0*	case	_	_	_	_	_	_
3	el	<0	det	_	_	_	_	_	_
4	texto	\0/0<0*	obl	_	_	_	_	_	_
5	,	>0*	punct	_	_	_	_	_	_
6	la	<0*	det	_	_	_	_	_	_
7	autora	\0<0	nsubj	_	_	_	_	_	_
8	habla	\0>0*/0	root	_	_	_	_	_	_
9	de	<0*	mark	_	_	_	_	_	_
10	lo	<0	det	_	_	_	_	_	_
11	que	<0	nsubj	_	_	_	_	_	_
12	son	<0	cop	_	_	_	_	_	_
13	las	<0	det	_	_	_	_	_	_
14	bellísimas	<0	amod	_	_	_	_	_	_
15	personas	\0>0	ccomp	_	_	_	_	_	_
16	.	>0*	punct	_	_	_	_	_	_


### 3- Limpiar las predicciones
1. Añadir indicadores de "principio de frase"
2. Añadir indicadores de "final de frase"
3. Limpiar otros campos

In [8]:

postprocess_path = "./demo_fix.labels"
is_new_sentence = True
with open(postprocess_path, "w") as f:
    for line in open("./demo.labels", "r"):
        fixed_line = []
        
        # add bos and eos
        if is_new_sentence:
            fixed_line.append(4*"-BOS-\t")
            is_new_sentence = False
            fixed_line = '\t'.join(fixed_line)
            f.write(fixed_line+"\n")
        
        if line == "\n":
            fixed_line.append(4*"-EOS-\t")
            is_new_sentence = True
            fixed_line = '\t'.join(fixed_line)
            f.write(fixed_line+"\n")
            f.write("\n")
            continue

        fixed_line = line.split('\t')[1:4]
        fixed_line.insert(1, "_")
        fixed_line = '\t'.join(fixed_line)
        f.write(fixed_line+"\n")

postprocess_path = "./demo_fix.labels"
postprocess_file = open(postprocess_path, "r")
postprocess = postprocess_file.read()
# read until empty line
postprocess = postprocess.split("\n\n")[0]
print(postprocess[:1000])

-BOS-	-BOS-	-BOS-	-BOS-	
-ROOT-	_	/0	-NOREL-
En	_	<0*	case
el	_	<0	det
texto	_	\0/0<0*	obl
,	_	>0*	punct
la	_	<0*	det
autora	_	\0<0	nsubj
habla	_	\0>0*/0	root
de	_	<0*	mark
lo	_	<0	det
que	_	<0	nsubj
son	_	<0	cop
las	_	<0	det
bellísimas	_	<0	amod
personas	_	\0>0	ccomp
.	_	>0*	punct
-EOS-	-EOS-	-EOS-	-EOS-	


### 4- Restaurar las relaciones de dependencias desde las abstracciones

Esto va a generar un archivo .labels con las representaciones intermedias que abstraen las dependencias del .conllu

In [ ]:
# python ~/CoDeLin/main.py DEPS DEC BRK_7B ./demo_fix.labels ./demo.conllu --multitask

In [37]:
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.deps_tree import D_Tree

# read conllu decoded file
conllu_file = "./demo.conllu"
trees = D_Tree.read_conllu_file(conllu_file)

# print sample
print(trees[0])
print(D_Tree.to_latex(trees[0]))

0	-ROOT-	_	-ROOT-	_	_	0	-NOREL-	_	_
1	En	_	_	_	_	3	case	_	_
2	el	_	_	_	_	3	det	_	_
3	texto	_	_	_	_	7	obl	_	_
4	,	_	_	_	_	3	punct	_	_
5	la	_	_	_	_	6	det	_	_
6	autora	_	_	_	_	7	nsubj	_	_
7	habla	_	_	_	_	0	root	_	_
8	de	_	_	_	_	14	mark	_	_
9	lo	_	_	_	_	14	det	_	_
10	que	_	_	_	_	14	nsubj	_	_
11	son	_	_	_	_	14	cop	_	_
12	las	_	_	_	_	14	det	_	_
13	bellísimas	_	_	_	_	14	amod	_	_
14	personas	_	_	_	_	7	ccomp	_	_
15	.	_	_	_	_	7	punct	_	_


\begin{dependency}[theme = simple]
\begin{deptext}[row sep=.25em, column sep=1.5em]
0 \& 1 \& 2 \& 3 \& 4 \& 5 \& 6 \& 7 \& 8 \& 9 \& 10 \& 11 \& 12 \& 13 \& 14 \& 15 \\ 
-ROOT- \& En \& el \& texto \& , \& la \& autora \& habla \& de \& lo \& que \& son \& las \& bellísimas \& personas \& . \\ 
\end{deptext}
\depedge{4}{2}{case}
\depedge{4}{3}{det}
\depedge{8}{4}{obl}
\depedge{4}{5}{punct}
\depedge{7}{6}{det}
\depedge{8}{7}{nsubj}
\depedge{1}{8}{root}
\depedge{15}{9}{mark}
\depedge{15}{10}{det}
\depedge{15}{11}{nsubj}
\depedge{15}{12}{cop}
\depedge{15}{13}{de